In [1]:
import numpy as np
from weighted_voronoi import weighted_voronoi

In [2]:
import arcpy

#from weighted_voronoi_main.weighted_voronoi_main import weightedVoronoi
from time import time

startTime = time()


In [3]:
pointSource =r'../test_data/Madagascar_Fake_Capital.shp'# arcpy.GetParameterAsText(0)
weightField = "POP"
vectorOutput = r'../test_data/Madagascar_Fake_Output.shp'
bufferDegrees = 5.0
cellSize = 0.008333333333333
distMethod = "vincenty"
smoothPolys = False

In [4]:
# Split out the destination file into path and name
vectorOutput = vectorOutput.replace("\\", "/")
vectorOutputPath = vectorOutput[:vectorOutput.rindex("/")]
vectorOutputName = vectorOutput[vectorOutput.rindex("/")+1:]

# Get the OID field name
oidField = arcpy.Describe(pointSource).OIDFieldName


def cleanup_scratch(refs):
    """
    Cleanup intermediate files that may be in scratch
    """
    # Cleanup intermediate files that may be around
    for r in refs:
        arcpy.Delete_management(r)


def cell_center_lat(row, initlat, ysize, numrows):
    """
    The latitude for the cell centers in a row
    """
    return initlat + (numrows - row - 1) * ysize


def cell_center_lng(col, initlng, xsize, numcols):
    """
    The longitude for the cell centers in a column
    """
    return initlng + (col) * xsize


# Get the spatial reference of the source (and dest)
spatialRef = arcpy.Describe(pointSource).spatialReference

# Split out the destination file into path and name
vectorOutputPath = vectorOutput[:vectorOutput.rindex("/")]
vectorOutputName = vectorOutput[vectorOutput.rindex("/")+1:]

# Intermediate features/rasters
vectorIntermediate = arcpy.env.scratchGDB + "/DVWPoly_vect_int"
rasterIntermediate = arcpy.env.scratchGDB + "/DVWPoly_rast_int"
rasterVals = arcpy.env.scratchGDB + "/DVWPoly_rast_vals"
rasterIds = arcpy.env.scratchGDB + "/DVWPoly_rast_ids"
extentBuffered = arcpy.env.scratchGDB + "/DVWPoly_extent_buffered"

# References to anything that might be in scratch
scratchRefs = [vectorIntermediate, rasterIntermediate, rasterVals,
               rasterIds, extentBuffered]

# Cleanup anything that may be leftover in scratch
cleanup_scratch(scratchRefs)

# Ensure that a scratch GDB is set
if arcpy.env.scratchGDB is None:
    arcpy.AddError("No scratch geodatabase is set")
    cleanup_scratch(scratchRefs)
    raise arcpy.ExecuteError

# Ensure that the point source is in a GCS
if spatialRef.type != "Geographic":
    arcpy.AddError("Must use a geographic coordinate system")
    cleanup_scratch(scratchRefs)
    raise arcpy.ExecuteError

# Get the extent of the point source and create as a polygon
sourceExtent = arcpy.Describe(pointSource).extent
extentPoints = arcpy.Array()
extentPoints.add(sourceExtent.lowerLeft)
extentPoints.add(sourceExtent.upperLeft)
extentPoints.add(sourceExtent.upperRight)
extentPoints.add(sourceExtent.lowerRight)
extentPoints.add(sourceExtent.lowerLeft)
sourceExtentPoly = arcpy.Polygon(extentPoints)

# Buffer the point source extent by bufferDegrees and update
# processing extent environment variable
arcpy.Buffer_analysis(sourceExtentPoly, extentBuffered,
                      bufferDegrees, "OUTSIDE_ONLY")
initExtent = arcpy.env.extent
arcpy.env.extent = arcpy.Describe(extentBuffered).extent

# Create a raster of weight values from weightField
arcpy.FeatureToRaster_conversion(
    in_features=pointSource,
    field=weightField,
    out_raster=rasterVals,
    cell_size=cellSize)
valRaster = arcpy.Raster(rasterVals)

# Create a raster of OIDs
arcpy.FeatureToRaster_conversion(
    in_features=pointSource,
    field=oidField,
    out_raster=rasterIds,
    cell_size=cellSize)
oidRaster = arcpy.Raster(rasterIds)

# Return to the initial processing extent
arcpy.env.extent = initExtent

# Create value and OID numpy arrays from rasters
arcpy.AddMessage("Creating value and OID numpy arrays")
valArray = arcpy.RasterToNumPyArray(valRaster, nodata_to_value=0)
oidArray = arcpy.RasterToNumPyArray(oidRaster, nodata_to_value=-1)


lowerLeft = arcpy.Point(oidRaster.extent.XMin, oidRaster.extent.YMin)
llCellX = oidRaster.extent.XMin + 0.5 * oidRaster.meanCellWidth
llCellY = oidRaster.extent.YMin + 0.5 * oidRaster.meanCellHeight
cellSizeY = oidRaster.meanCellHeight
cellSizeX = oidRaster.meanCellWidth
rasterHeight = int(oidRaster.height)
rasterWidth = int(oidRaster.width)

arcpy.AddMessage("Rasters height: %d" % rasterHeight)
arcpy.AddMessage("Rasters width: %d" % rasterWidth)

del valRaster
del oidRaster


In [5]:
# Arrays of the cell center latitudes for each row and longitudes for
# each column
arcpy.AddMessage("Determine lat for each row, lng for each column")
rowLats = np.array(
                    [cell_center_lat(x, llCellY, cellSizeY, rasterHeight)
                     for x
                     in range(rasterHeight)])

colLngs = np.array(
                    [cell_center_lng(y, llCellX, cellSizeX, rasterWidth)
                     for y
                     in range(rasterWidth)])

# Get the row/col refs for point locations in the raster and build a
# list of points including OID and coordinates
arcpy.AddMessage("Building up points to check")
pointLocations = zip(*map(list, np.where(oidArray != -1)))
#checkPoints = [
#    {
#        "cell": l,
#        "coords": (rowLats[l[0]], colLngs[l[1]]),
#        "oid": oidArray[l],
#        "val": valArray[l]
#    }
#    for l in pointLocations
#]

oidUse = oidArray[oidArray!=-1]
valUse = valArray[oidArray!=-1]
latsUse = np.asarray([rowLats[l[0]] for l in pointLocations])
lonsUse = np.asarray([colLngs[l[1]] for l in pointLocations])
del oidArray
del valArray
# the cython has been compiled for int32 specifically
assignmentsDType = np.int32
vCalc = weighted_voronoi(rowLats, colLngs, oidUse, valUse, latsUse, lonsUse)
assignments = np.asarray(vCalc.calculate())

In [6]:
# Save the array back to a raster
arcpy.AddMessage("Assignments completed, saving to raster")
tempRast = arcpy.NumPyArrayToRaster(assignments, lowerLeft,
                                   cellSizeX, cellSizeY)
tempRast.save(rasterIntermediate)
del tempRast

# Define the raster projection
arcpy.DefineProjection_management(rasterIntermediate, spatialRef)

# Create an intermediate polygon feature class in scratch because
# AlterField_management doesn't appear to work with the output shapefile.
arcpy.AddMessage("Polygonizing raster")
arcpy.RasterToPolygon_conversion(
    in_raster=rasterIntermediate,
    out_polygon_features=vectorIntermediate,
    simplify=("SIMPLIFY" if smoothPolys else "NO_SIMPLIFY"),
    raster_field="Value")

# Delete the kinda-duplicative Id field
arcpy.AddMessage("Cleaning up polygon fields")
arcpy.DeleteField_management(vectorIntermediate, "Id")

# Rename GRIDCODE field to align with the original OID fieldname, e.g. ORIG_FID
arcpy.AlterField_management(vectorIntermediate,
                            "GRIDCODE",
                            str("ORIG_" + oidField))

# Copy the intermediate vector output to its final destination
arcpy.AddMessage("Saving final polygon output to {}".format(vectorOutput))
arcpy.FeatureClassToFeatureClass_conversion(
    in_features=vectorIntermediate,
    out_path=vectorOutputPath,
    out_name=vectorOutputName
  )

# Cleanup intermediate files
cleanup_scratch(scratchRefs)


In [2]:
#oidUse
oidUse = np.array([1,2,3,4,5,6,7,8])

In [3]:
#valUse
valUse = np.array([  68973.,  124745.,  160963.,  128893.,   54578.,  127628.,
         94394.,   53067.])

In [4]:
#latsUse
latsUse = np.array([-12.31252035, -15.71252035, -18.16252035, -18.91252035,
       -19.67085368, -21.44585368, -23.34585368, -25.16252035])

In [5]:
#lonsUse
lonsUse=np.array([ 49.28752035,  46.31252035,  49.37918701,  47.51252035,
        47.31252035,  47.06252035,  43.66252035,  46.07918701])

In [6]:
colLngs = np.arange(46,49,0.008333333333333)

In [7]:
rowLats = np.arange(-12,-26,-0.008333333333333)

In [13]:
rowLats

array([-12.        , -12.00833333, -12.01666667, ..., -25.98333333,
       -25.99166667, -26.        ])

In [8]:
assignments = weighted_voronoi(rowLats, colLngs, oidUse, valUse, latsUse, lonsUse)

In [9]:
r = assignments.calculate()

In [11]:
np.asarray(r)

array([[2, 2, 2, ..., 1, 1, 1],
       [2, 2, 2, ..., 1, 1, 1],
       [2, 2, 2, ..., 1, 1, 1],
       ..., 
       [8, 8, 8, ..., 6, 6, 6],
       [8, 8, 8, ..., 6, 6, 6],
       [8, 8, 8, ..., 6, 6, 6]])